In [1]:
!pip -q install imageio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Manipulação de arquivos
import os
import csv
import random

# controle de data e tempo
import time
from datetime import datetime

#Manipulação de imagens
import cv2
import imageio
from IPython.display import Image
from PIL import Image as pil

# Ciencia de dados
import numpy as np
import pandas as pd

# Apresentação
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import glob

# Bloquear avisos de cluster
import warnings
warnings.filterwarnings("ignore")

# Funções sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, silhouette_score

# funções de cluster
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture

#Salvar os modelos
from joblib import dump, load

# Configuração do repositorio do Driver

1.   Criar uma pasta global - Nome a ser definido pelo usuário
2.   Dentro da pasta do  item (1) criar as pastas
* 2.1 - Imagens: Diretorios com as imagens coletadas
** 2.1.1 - Analise: Imagem que compõem o dataset para analise
** 2.1.2 - treino: Imagem que compõem algoritimos de ML
** 2.1.3 - validacao: Imagem para validar modelos
* 2.2 - Modelos: Diretorio para armagenamentos dos modelos treinados
* 2.3 - Logs: Diretorio para armazenamento de logs
* 2.4 - Saidas:  Diretorio para armazenamento de saidas CSV


In [11]:
#======================================================================
#          Variaveis Globais  serão utilizados em todo o código
#======================================================================
# Caminho do driver de cada participante aonde esta as imagens
path_Leandro = 'drive/MyDrive/dataSets_Lab/'
path_Andre   = '/content/drive/MyDrive/Estudo/Mestrado Data Science/TFM/'
path_Vitoria = ''
path_Bruno   = ''
path_Patrick = '/content/drive/MyDrive/Nuclio_School/TFM/dataSets_Lab/'
path_Vagner  = ''

# selecionar o path do usuário
#path = path_Leandro
path = path_Andre

# Gerar a lista de diretórios e arquivos
dir_list = {'2'    :f'{path}Imagens',
            '2.1.1':f'{path}Imagens/analise/',
            '2.1.2':f'{path}Imagens/treino/',
            '2.1.3':f'{path}Imagens/validacao/',
            '2.2'  :f'{path}Modelos/',
            '2.3'  :f'{path}Logs/',
            '2.4'  :f'{path}Saidas/'
           }

model_name  = {'1':'espectro','2':'menoDesmat','3':'maioDesmat'}
saidas_name = {'1':'analise.csv','2':'load_image.pkl'}
logs_name   = {'1':'ana_classificador.csv'}

# Funções globais que serão usadas em varios trechos do código fonte.  

In [12]:
#========================================================
#          Função para apresentar a imagem
#========================================================
def apresentar(img):
  cv2_imshow(img)


#========================================================
#          Função para graver as imagens
#========================================================
def gravar(filename, img):
  cv2.imwrite(filename, img)


#========================================================
#               Função para abrir a imagem
#========================================================
def openImageCinza(path):
    img  = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.resize(img, (255,255), interpolation = cv2.INTER_AREA)



def openImageColorida(path):
    imgc  = cv2.imread(path)
    return cv2.resize(imgc, (255,255), interpolation = cv2.INTER_AREA)

#========================================================
#        Função para gerar a lista com os arquivos
#========================================================
def getFiles(dir):
    listFiles = []
    for currentFolder, subFolder, files in os.walk(dir):
        for file in files:
            arquivo = dir+file
            if os.path.exists(arquivo) and arquivo not in listFiles:
                listFiles.append([arquivo,file])
    return listFiles


#===============================================================
#        Função para gerar um arqui .csv com dados das imagens
#==============================================================
def gerarArquivo(valor, path):
  # Verificar se o arquivo já existe
  continuacao = ""
  if os.path.exists(path):
    for linha in open(path, "r"):
      continuacao += linha

  # Escrever
  with open(path, "w") as arquivo:
    if len(continuacao) > 0:
      arquivo.write(continuacao+'\n')
    line = ';'.join(map(str, valor))
    arquivo.write(line)
    arquivo.close()

#===============================================================
#        Função de validação do arquivo
#==============================================================
def clearCvs():
  ds_carregado = pd.read_csv(dir_list['2.4']+saidas_name['2'], delimiter=';')
  return ds_carregado



#===============================================================
#        Função que gerar os modelos de claficação
#==============================================================
def model_generator(df,x_col, y_col, path):
  nom_model = ""
  vd_score = 0
  espc_model = None
  data_atual = datetime.now()
  data_atual = data_atual.strftime("%d-%m-%Y %H:%M")
  saidas = [f'Treinamento realizado em {data_atual}']

  # Dividir o conjunto de dados em features (entradas) e target (saída)
  X = df[x_col]
  y = df[y_col]

  # Dividir os dados em  treinamento e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  parametros = [int(x * 3) for x in range(7)]

  for lup in  parametros:
    # Inicializar e treinar os modelos de clustering
    kmeans = KMeans(n_clusters=lup, random_state=42)
    dbscan = DBSCAN(eps=(lup/10), min_samples=lup)
    gmm = GaussianMixture(n_components=lup, random_state=42)
    rfc = RandomForestClassifier(n_estimators=int(lup*10), random_state=42)

    # Lista de modelos
    modelos = [('K-Means', kmeans),
            ('DBSCAN', dbscan),
            ('RandomForest', rfc),
            ('Gaussian Mixture', gmm)]

    # Avaliar os modelos
    for nome, modelo in modelos:
      start = time.time()

      try:
        # Ajustar o modelo
        if nome != 'RandomForest':
          labels = modelo.fit_predict(X)
          score = silhouette_score(X, labels)
          mensagem =  f'{nome}: Índice de Silhueta = {score} tempo {(time.time() - start)} com cluster {lup}'
        else:
          modelo.fit(X_train, y_train)
          y_pred = modelo.predict(X_test)
          score = accuracy_score(y_test, y_pred)
          mensagem = f'{nome}: accuracy_score = {score} tempo {(time.time() - start)} com estimators {int(lup * 10)}'

        # Verificar se o modelo possui a melhor performance
        if vd_score < score:
          vd_score = score
          espc_model = modelo
          nom_model = f'{nome} com {lup} cluster '
          if nome == 'RandomForest':
            nom_model = f'{nome} com {int(lup * 10)} estimators'

      except:
        if nome != 'RandomForest':
          mensagem = f'{nome}: não aplicavel ao dataset com {lup} cluster'
        else:
          mensagem = f'{nome}: não aplicavel ao dataset com {int(lup * 10)} estimators'

      print(mensagem)
      saidas.append(mensagem)

  mensagem = f'====================== (O modelo carrega foi {nom_model}) ======================='
  print(mensagem)
  saidas.append(mensagem)

  # Gravar saidas
  saidas = pd.DataFrame(saidas)
  saidas.to_csv(dir_list['2.3']+path+'.csv', sep=';', index=False)
  dump(espc_model, dir_list['2.2']+path+'.joblib')


#1 - Inicio da execução

In [6]:
#lista de saida
lista =  []

# Pega o caminho das imagens
files_path = getFiles(dir_list['2.1.1'])

# fragmentar o arquivo para extrair informações
for item in files_path:

  #Sepra caminho do nome da imagem
  path, info = item

  # Pega Ano, regisão, latitude e longitude
  #ano, reg, lat, log = info.split('.')[0].split("_")
  reg, ano = info.split('.')[0].split("_")

 # Acrescentar intem na lista
  lista.append([ano, reg, path])

#Gerar um datasets para facilitar o acesso de informações
df_lista = pd.DataFrame(lista, columns=['ano', 'reg', 'path'])

#Apresentar metricas de validação do datasets
print("=========== (Cada arquivo deve ser uma linha no dataset) ==============")
print(f'Total de arquivo:{len(files_path)} carregados no dataset:{len(df_lista)}')

#verifiar as 5 primeiras linha do conjunto de dados
df_lista.head(5)

=========== (Cada arquivo deve ser uma linha no dataset) ==============
Total de arquivo:0 carregados no dataset:0


,ano,reg,path


#2 - Analise das imagens  


*   Analisar a imagem em aspextos de pixels
*   Classificar os pixels em apenas 2 cores 0 -  Verde e 255 - Ausencia de verde




In [7]:
#===============================================================================
#                      Função para classificar as cores
#===============================================================================

#*************************(Função com RN )**************************************
def espect_v1(path):
  img = openImageCinza(path)
  #Gerar uma cópia preta e branca sendo: preta - mata e branca - desmatamento
  for i in range(len(img)):
    for j in range(len(img[i])):
      vl_px = img[i][j]
      if vl_px < 30:
        img[i][j] = 255
      elif vl_px < 70:
        img[i][j] = 0
      else:
        img[i][j] = 255
  return img


#***************(Função com algoritomo de ML para classificação )***************
def espect_v2():
  #=================( Variaveis de contexto )=======================
  img = cv2.imread(dir_list['2.1.2']+'1_2010.jpg')
  nom_model = ""
  vd_score = 0
  espc_model = None
  data_atual = datetime.now()
  data_atual = data_atual.strftime("%d-%m-%Y %H:%M")
  saidas = [f'Treinamento realizado em {data_atual}']

  #Crop da imagem
  fx      = img[220:400]

  #Gravar trecho recortado
  gravar(dir_list['2.1.3']+'crop_0.jpg', fx)

  figs    = [
               [fx[:,:100],0],
               [fx[55:90,1100:1300],0],
               [fx[55:90,800:1050],255],
               [fx[60:70,1320:1340],255]
             ]

  # Montagem da estrutura de dados
  cr = 1
  matriz = []
  for cor in figs:
    tobe = cor[1]
    asis = cor[0]
    gravar(dir_list['2.1.3']+f'crop_{cr}.jpg', asis)
    for i in asis:
      for j in i:
        matriz.append([j[0],j[1],j[2],tobe])
    cr += 1
  df = pd.DataFrame(matriz,columns=["r","g","b","saida"])
  model_generator(df,['r', 'g', 'b'], 'saida', model_name['1'])

#========================================================
# Função para apresentar a imagens com seu spectro
#========================================================
def analiseDataSet(list):
  #Gerar o caminho do dataset existente
  path =  dir_list['2.4']+saidas_name['2']

  #Carregar o modelo de classificação de cores
  modelo = load(dir_list['2.2']+model_name['1']+'.joblib')

  # Gerar o nome das colunas
  columns = ['file', 'linha']
  for i in range(255):
      columns.append(f'px{i}')

  for item in list:
    try:
      #Recuperar o nome do arquivo
      info_path = item.split("/")
      nome = f'{info_path[len(info_path)-1]}'
      classificar = False

      #ler a imagem
      try:
        img = openImageColorida(item)
      except:
        img =  pil.open(item)
        img = np.asanyarray(img)
        img = cv2.resize(img, (255,255), interpolation = cv2.INTER_AREA)

      #Carregar o datasets gravado
      existente = None
      if os.path.exists(path):
        existente = pd.read_pickle(path)
        if (not existente['file'].isin([nome]).any() ):
          classificar = True
      else:
        classificar = True

      #verificar se a imagem já foi carregada no dataset
      if classificar:
        matriz = []
        #Classificar os pixels
        for i in range(len(img)):
          gravar = [nome,i]
          for j in range(len(img[i])):
            predito =  modelo.predict([img[i][j]])[0]
            gravar.append(predito)
          matriz.append(gravar)

        #Transforma a matriz em pandas dataframe
        df_pronto = pd.DataFrame(matriz, columns=columns)

        #Salvar o dataSet
        if existente is not None:
          df_pronto = pd.concat([existente, df_pronto])

        df_pronto.to_pickle(path)
        print(f'Item {item} incluido com sucesso no dataset!!')
      else:
        print(f'Item {item} já existente no dataset!!')
    except BaseException as e:
      print(e)
      print(f'Falha no arquivo {item}')
  return pd.read_pickle(path)



#========================================================
# Função para gerar o gift
#========================================================
def gift(ds_carregado):
     # recuperar o nome do arquivo
     nomes = ds_carregado.file.drop_duplicates()

     # armazenar os frames
     frames = []

     for nome in nomes:
      # Verificar se a imagem treinada ainda existe
      path = dir_list['2.1.1']+nome[0]

      if os.path.exists(path):
        # Imagem original
        img_original = openImageCinza(path)

        # Garregar as imagens salvas
        try:

          espectro = ds_carregado[ds_carregado.file == nome]
          espectro = espectro.drop(['file','linha'], axis=1)
          espectro = espectro.values.astype(np.uint8)
          img_es = cv2.cvtColor(espectro, cv2.COLOR_GRAY2BGR)
          img_es = cv2.cvtColor(img_es, cv2.COLOR_BGR2GRAY)

          # Juntar as duas imagens original e espectro em uma só
          frames.append(cv2.hconcat((img_original, img_es)))

        except:
          print("")

        # Montar uma imagem gift animado para analise dos detalhes
        with imageio.get_writer('animacao.gif', mode = 'I', duration = 5.0) as writer:
          for img in frames:
            img_animada = cv2.resize(img, (600, 300))
            writer.append_data(img_animada)


     return  frames

#========================================================
# Função para reconstruir a imagem a partir do dataset
#========================================================
def rebuild(ds_img):
    espectro = ds_img.drop(['file','linha'], axis=1)
    img_es = cv2.cvtColor(espectro.values.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    return cv2.cvtColor(img_es, cv2.COLOR_BGR2GRAY)

#===========================================================================================
# Função para apresentar os resultado do treinamento dos modelo de classificação de pixels
#==========================================================================================
def acurracy_ModelEspec(path):
  metrics = pd.read_csv(path, delimiter=";")
  for x in metrics.iloc():
    print('=  {}'.format(x['0']))

In [8]:
#Carregar o data set com as imagens já classificadas
ds_carregado = analiseDataSet(df_lista.path)
#Classificar os pixels


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Estudo/Mestrado Data Science/TFM/Modelos/espectro.joblib'

In [ ]:
#===============================================================================
#           Compara as funções que geram a imagem de espectro
#===============================================================================

# Imagem de teste
path = dir_list['2.1.1']+'1_2014.jpg'

# Carregar o melhor modelo
img_if  =  espect_v1(path)
if os.path.exists(dir_list['2.2']+model_name['1']+'.joblib') == False:
  espect_v2()

#Carregar as imagem (original, teste, com)
teste    = openImageColorida(path)
teste    = cv2.resize(teste, (255,255), interpolation = cv2.INTER_AREA)
img_ml   = rebuild(ds_carregado[ds_carregado.file == '1_2014.jpg'])
original = openImageCinza(path)


ap = cv2.hconcat((original,img_if,img_ml))
print("======================== (Apresentar os resultados do treinamento do modelo ) ============================= ")
print("=  ==================================================================================")
acurracy_ModelEspec(dir_list['2.3']+model_name['1']+'.csv')
print("\n Esqueda: original | Meio: Aplicação IF | Direita: Aplicação ML ")
apresentar(ap)

In [ ]:
#========================================================
# Analisar as imagens de crop
#========================================================
list_crop = getFiles(dir_list['2.1.3'])
for crop in list_crop:
  path = crop[0]
  print(f'imagem {crop[1]}')
  try:
    apresentar(cv2.imread(path))
  except:
    print(f'Erro na apresentação do arquivo {path}')

In [ ]:
ds_carregado = ds_carregado.drop('reg', axis=1)
ds_carregado['reg'] = ds_carregado['file'].iloc[0][:1]


In [ ]:
#============= ( Analise da imagens para montagem do data sets ) ===============
# Gerar o arquivo para armazenar os dados ja classificados  pela ML
if os.path.exists(dir_list['2.4']+saidas_name['2']) == False:
  print("Aqui")
  cabeca = ['file','linha']
  for i in range(255):
    cabeca.append(f'px{i}' )
    gerarArquivo
  gerarArquivo(cabeca, dir_list['2.4']+saidas_name['2'])

# Chamada da função para analise
start = time.time()
print(f'tempo de execução {(time.time() - start)}')
gift(ds_carregado)
frames  = gift(ds_carregado)
# Apresentar com gift
Image(open('animacao.gif', 'rb').read())

In [ ]:
#============= ( Apresentar as imagens de forma separada em lista ) ============
x = 1
for i in frames:
  print(f"====== {x} =======")
  apresentar(i)
  x += 1

# Conclusão da analise das imagens
1 - Como pode ser observado a classificação das imagens em que, a área de mata fica com o rotulo 0 e as áres de desmatamento ficam com o rotulo 255.
Desta forma é possível identificar as zonas de desmatamento.  

# 2 - Contrução do dataSet

# Dicionario do data SET
'data': Indice do dataset sempre 01 de Janeiro do ano extraido no nome da imagem  
'ano': Ano extraido no nome da imagem

'reg': Região extraido no nome da imagem

'lat': Latitude extraido no nome da imagem

'log': Longitude extraido no nome da imagem

'des_NO': Quantidade de pix classificado com desmatamento na região NO

'mat_NO': Quantidade de pix classificado com Mata na região NO

'vld_NO': Somatorio dos valores pixels classificado com desmatamento na região NO

'vlm_NO': Somatorio dos valores pixels classificado com mata na região NO

'des_NE': Quantidade de pix classificado com desmatamento na região NE

'mat_NE': Quantidade de pix classificado com Mata na região NE

'vld_NE': Somatorio dos valores pixels classificado com desmatamento na região NE

'vlm_NE': Somatorio dos valores pixels classificado com mata na região NE

'des_SO': Quantidade de pix classificado com desmatamento na região SO

'mat_SO': Quantidade de pix classificado com Mata na região SO

'vld_SO': Somatorio dos valores pixels classificado com desmatamento na região SO

'vlm_SO': Somatorio dos valores pixels classificado com mata na região SO

'des_SE': Quantidade de pix classificado com desmatamento na região SE

'mat_SE': Quantidade de pix classificado com Mata na região SE

'vld_SE': Somatorio dos valores pixels classificado com desmatamento na região SE

'vlm_SE': Somatorio dos valores pixels classificado com mata na região SE

In [ ]:
def dataSet_Gerador(ds_carregado):
  df_saida = []


  #Gerar uma lista com o nome dos arquivos
  list_file = ds_carregado.file.drop_duplicates()


  for file in list_file:

    #Recuperar o ano
    nome = file.split("_")
    reg = nome[0]
    ano = nome[1][:4]


    # Recuperar os dados da imagem para calcular as posições
    espectro = ds_carregado[ds_carregado.file == file]
    espectro = espectro.drop(['file','linha'], axis=1)
    img_es = cv2.cvtColor(espectro.values.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    img = cv2.cvtColor(img_es, cv2.COLOR_BGR2GRAY)

    centro = int(img.shape[0]/2)

    # Nodeste  valores
    des_NO,mat_NO,vld_NO,vlm_NO = 0,0,0,0

    # Noroeste valores
    des_NE,mat_NE,vld_NE, vlm_NE = 0,0,0,0

    # Sudoeste Valores
    des_SO,mat_SO,vld_SO,vlm_SO = 0,0,0,0

    # Sudeste valores
    des_SE,mat_SE,vld_SE, vlm_SE = 0,0,0,0


    #Classificar os pixels
    for i in range(len(img)):
      for j in range(len(img[i])):

        # Pegar o valor do pixel
        vl_px = img[i][j]

        #NO
        if i <= centro and j <= centro:
          if vl_px == 0:
            mat_NO += 1
            vlm_NO += vl_px
          else:
            des_NO += 1
            vld_NO += vl_px

        #NE
        elif i <= centro and j > centro:
          if vl_px == 0:
            mat_NE += 1
            vlm_NE += vl_px
          else:
            des_NE += 1
            vld_NE += vl_px

        #SO
        elif i > centro and j <= centro:
          if vl_px == 0:
            mat_SO += 1
            vlm_SO += vl_px
          else:
            des_SO += 1
            vld_SO += vl_px

        #SE
        else:
          if vl_px == 0:
            mat_SE += 1
            vlm_SE += vl_px
          else:
            des_SE += 1
            vld_SE += vl_px

    # Adicionar os valores a matriz
    df_saida.append([f"{ano}-01-01", ano, reg,
                    des_NO, mat_NO, vld_NO, vlm_NO,
                    des_NE, mat_NE, vld_NE, vlm_NE,
                    des_SO, mat_SO, vld_SO, vlm_SO,
                    des_SE, mat_SE, vld_SE, vlm_SE
                    ])

  # Gerar nome das colunas do data frame
  columns = ["data", 'ano', 'reg','des_NO','mat_NO','vld_NO',
             'vlm_NO','des_NE', 'mat_NE', 'vld_NE', 'vlm_NE','des_SO',
             'mat_SO', 'vld_SO', 'vlm_SO','des_SE','mat_SE','vld_SE','vlm_SE']

  return pd.DataFrame(df_saida, columns=columns)

In [ ]:
# Gerar o data set
df = dataSet_Gerador(ds_carregado)

#Ordenar o dataset  por ano e regiao
df.sort_values(by=['ano','reg'],inplace=True)

#passar a coluna data para datetime
df['data'] = pd.to_datetime(df.data)

# Colocar a coluna data como indice
df.set_index('data', inplace=True)

#Ajustar a coluna ano para int facilitar a analise
df['ano'] = df.ano.astype(int)

#apresentar as 5 primeiras linhas
df.head(5)

if os.path.exists(dir_list['2.4']+saidas_name['1']):
  os.remove(dir_list['2.4']+saidas_name['1'])

# Salvar o DataFrame em um arquivo CSV separado por ponto e vírgula
df.to_csv(dir_list['2.4']+saidas_name['1'], sep=';', index=False)

# 3 - Analise do dataset


In [ ]:
#===============================================================================
#           Função que gerar uma paleta de cores em hexadecimal
#===============================================================================
def colores(qtd):
  list_hex =[]
  for i in range(qtd):
    color = np.random.choice(range(256), size=3)
    color = ["#" + "".join([random.choice("0123456789ABCDEF") for j in range(6)])][0]
    if color not in list_hex:
      list_hex.append(color)
  return list_hex

In [ ]:
#Inicialmente plotaremos os topicos
df.plot(figsize=(15,6))
# No grafico plotado é possivel perceber que as metricas relacionada a quantidade de mata na região analisada
# Decer conforme é aumentado o percentual de desmatamente.
# É possivel ver uma linearidade entre os dados em uma escala temporal.

In [ ]:
# Gerar Dicionarios de cores  para os graficos
qtd_regioes =  [ int(x[0]) for x in df.reg.drop_duplicates()]
list_hex =colores(len(qtd_regioes))

In [ ]:
# Vamos analise a componente de tendencia do desmatamento nas imagen geral.
plt.figure(figsize=(15, 10))
i = 0
for reg in qtd_regioes:
  df_g = df[df.reg == str(reg)]
  df_tendencia = df_g.des_NO+ df_g.des_NE+ df_g.des_SO + df_g.des_SE
  plt.plot(df_tendencia,  color=list_hex[i], label=f'{reg}')
  i += 1
plt.legend(loc='best')
#É possivel verificar que existe um linearidade no aumento do desmatamento (Leandro)


In [ ]:
# Vamos analise com um informação real.  As eleições municipais aconteceram no
# Anos 1994 até 2020 de 4 em 4 anos .
intervalos = int(df.ano.max()) - int(df.ano.min())
if intervalos % 4:
  print("Existe periodo fora das eleições é preciso ajusta")

df_filter = df[(df.ano >= 1992) & (df.ano < 2021)]
print(f"Periodo inicial {df_filter.ano.min()} e maximo {df_filter.ano.max()}")

i = 0
for reg in qtd_regioes:
  df_re = df_filter[df_filter.reg == f'{reg}']
  df_g = df_re.des_NO + df_re.des_NE+ df_re.des_SO + df_re.des_SO
  plt.figure(figsize=(15, 10))
  plt.plot(df_g.rolling(4).mean(), color=list_hex[i], label=f'{reg}')
  i += 1
plt.legend(loc='best')

In [ ]:
# Analise o desmatamento no primeiro ano apos a eleições municipais

# Gerar o datasets só com o periodo da analise
periodos = np.arange(2000, 2024, 4, dtype=None)
df_ano = df[df.ano == 2000]

for  i in periodos:
  df_p = df[df.ano == i]
  df_ano = pd.concat([df_ano, df_p], ignore_index=True)

# Ordenar pela data
df_ano.sort_values(by=['ano'], inplace=True)


# checar se o data set possui apenas os anos corretos
ano_df = df_ano.ano.drop_duplicates()

# colocar a coluna ano como idex
df_ano.set_index('ano', inplace=True)

# Gerar data set
#df_tendencia1 = df_ano.des_NO + df_ano.des_NE+ df_ano.des_SO + df_ano.des_SE
lista_c = colores(int(len(qtd_regioes)*4))


# Apresentar
plt.figure(figsize=(8, 6))
i = 0
for reg in qtd_regioes:
  df_g = df_ano[df_ano.reg == f'{reg}']
  plt.plot(df_g.des_NO, color=lista_c[(int(i))], label=f'reg{reg} NO')
  plt.plot(df_g.des_NE, color=lista_c[(int(i+1))], label=f'reg{reg} NE')
  plt.plot(df_g.des_SO, color=lista_c[(int(i+2))], label=f'reg{reg} SO')
  plt.plot(df_g.des_SE, color=lista_c[(int(i+3))], label=f'reg{reg} SE')

plt.legend(loc='best')

In [ ]:
# Analise o desmatamento no primeiro ano apos a eleições municipais

# Gerar o datasets só com o periodo da analise
periodos = np.arange(1998, 2022, 4, dtype=None)
df_ano = df[df.ano == 1994]
for  i in periodos:
  df_p = df[df.ano == i]
  df_ano = pd.concat([df_ano, df_p], ignore_index=True)

# # Ordenar pela data
df_ano.sort_values(by=['ano'], inplace=True)


if i not in periodos and i != 1993:
  print(f"{i} esté valor esta fora do periodo de apuração")

if len(df_ano) != (len(periodos) +1):
  print(f" Existe divergencia entre os dados {len(df_ano)} {(len(periodos) +1)}")

# colocar a coluna ano como idex
df_ano.set_index('ano', inplace=True)

# Gerar data
for reg in qtd_regioes:
  df_g = df_ano[df_ano.reg == f'{reg}']
  df_tendencia = df_g.des_NO + df_g.des_NE+ df_g.des_SO + df_g.des_SO

# Apresentar
df_tendencia.plot(kind ='bar')

# Verificar em qual direção esta avançando o desmatamento

In [ ]:
# Gerar data set
df_tendencia = df[["des_NO","des_NE","des_SO","des_SE"]]
# Apresentar
df_tendencia.plot(kind ='bar', figsize=(15,6))
# De acordo com o grafico a area do municipio que mais possui desmatamento é a area
# Superior Esquerda representada pelo região Noroeste.

# Andre


#Leandro

In [ ]:
# Composição da função que mapeia as posições da regiões mais desmatados
def posicao(df, ano, reg):
    # Separa os periodos de apuração
    com_df = df[(df['ano'] == ano) & (df['reg'] == str(reg))]

    if not com_df.empty:
        maior_desmatamento = com_df[['des_NO', 'des_NE', 'des_SO', 'des_SE']].max(axis=1)
        menor_desmatamento = com_df[['des_NO', 'des_NE', 'des_SO', 'des_SE']].min(axis=1)

        com_df['Maior_Desmatamento'] = maior_desmatamento
        com_df['Menor_Desmatamento'] = menor_desmatamento

        # Criar as colunas de maior e menor desmatamento
        com_df['Região_Maior_Desmatamento'] = com_df[['des_NO', 'des_NE', 'des_SO', 'des_SE']].idxmax(axis=1).str[4:]
        com_df['Região_Menor_Desmatamento'] = com_df[['des_NO', 'des_NE', 'des_SO', 'des_SE']].idxmin(axis=1).str[4:]

    return com_df

In [ ]:
# juntar o data setes
df_classificado = None

#Gerar um lista agrupada por ano e regiões
lista_faixa = df[['ano','reg']].drop_duplicates()

#Identificar as regiões com maior e menor desmatamento
for i in lista_faixa.iloc():
  df_stage =  posicao(df, i['ano'],i['reg'])
  df_classificado = pd.concat([df_classificado, df_stage])

# Elencar as colunas necessárias para o treinamento da proxima faze
colunas_necssarias =  ['ano','reg','des_NO','des_NE', 'des_NO','des_SE',
                       'Maior_Desmatamento', 'Menor_Desmatamento']

df_classificado = df_classificado[colunas_necssarias]


In [ ]:
model_generator(df_classificado,['ano','reg','des_NO','des_NE', 'des_NO','des_SE'], 'Menor_Desmatamento', model_name['3'])


In [ ]:
model_generator(df_classificado,['ano','reg','des_NO','des_NE', 'des_NO','des_SE'], 'Maior_Desmatamento', model_name['2'])

In [ ]:
 df_imagens  =  df_lista[df_lista.ano == df_lista.ano.max()]
for paths in df_imagens.iloc():
  img = openImageColorida(paths['path'])
  centro = int(255 /2)
  no = cv2.resize(img[:centro,:centro], (127,127), interpolation = cv2.INTER_AREA)
  ne = cv2.resize(img[:centro,centro:], (127,127), interpolation = cv2.INTER_AREA)
  so = cv2.resize(img[centro:,:centro], (127,127), interpolation = cv2.INTER_AREA)
  se = cv2.resize(img[centro:,centro:], (127,127), interpolation = cv2.INTER_AREA)

  zr = np.zeros(no.shape[:2], dtype = "uint8")

  r1,g1,b1 = cv2.split(no)
  r2,g2,b2 = cv2.split(se)
  no = cv2.merge([zr, zr,r1])
  se = cv2.merge([b2, zr,zr])
  fx1 = cv2.hconcat([no, ne])
  fx2 = cv2.hconcat([so, se])
  fn = cv2.vconcat([fx1, fx2])
  apresentar(fn)




In [ ]:
(255 * 255) / (30*30)

In [ ]:
30*30